# 3. DESGLOSE TEMPORAL Y ASIGNACIÓN ADMINISTRATIVA #

- Separación de fecha en: año, mes y dia
- Unión espacial con municipios
- Normalización de nombres administrativos

## 3.1 Extraer Fecha de Base de Datos FIRMS para los Años de Estudio ##

Preprocesamiento básico de una base de datos FIRMS en formato Excel, buscando derivar explícitamente el año y el mes a partir del campo de fecha de adquisición (ACQ_DATE).

### 3.1.1 Parte 1 ###

Importanción de librerias a utilizar, definción de ruta del archivo y lectura dentro de un Dataframe. Convierte la columna ACQ_DATE de texto a tipo datetime. Se extrae el año y mes y se verifica el resultado obtenido. Unificando todos los años para FIRMS en un documento excel que posteriormente se convertira a formato shape llamado "puntos_unificados_FIRMS_25_11_24_fecha_desglosada.shp".

In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import os
from src.firms_excel import setup_logger, process_firms_excel

in_xlsx = r"C:\Users\Equipo\Tesis\DATOS_25_06_24\FIRMS2023_25_06_26.xlsx"
out_xlsx = r"C:\Users\Equipo\Tesis\DATOS_25_06_24\FIRMS2023_25_06_26_V2.xlsx"
log_path = os.path.join(os.path.dirname(out_xlsx), "log_firms_excel.txt")

logger = setup_logger(log_path)

summary = process_firms_excel(
    in_path=in_xlsx,
    out_path=out_xlsx,
    date_col="ACQ_DATE",
    date_format="%Y/%m/%d",
    version_out=True,
    logger=logger
)

print(summary)

ExcelSummary(n_rows_in=839, n_rows_out=839, n_null_dates=0, out_path='C:\\Users\\Equipo\\Tesis\\DATOS_25_06_24\\FIRMS2023_25_06_26_V2.xlsx')


In [20]:
import pandas as pd
df_check = pd.read_excel(summary.out_path)
df_check[["ACQ_DATE","ano_FIRMS","mes_FIRMS"]].head()

,ACQ_DATE,ano_FIRMS,mes_FIRMS
0,2023-02-02,2023,2
1,2023-02-03,2023,2
2,2023-09-28,2023,9
3,2023-09-28,2023,9
4,2023-09-28,2023,9


## 3.2 Asignar Municipios al Total de Puntos Calientes Derivados de FIRMS ## 

Asignación espacial de puntos FIRMS a municipios, mediante una unión espacial (spatial join) entre un shapefile de polígonos municipales y un shapefile de puntos de incendios.

### 3.2.1 Parte 1  ###

Cargar librerias. Lectura de capas de entrada y armonización de CRS. Verificación de columnas y verificación de existencia de los campos minimos para identificación de departamentos y municipios. Normalización de columnas de interes. Verificación de geometrias de los geodataframes, unión espacial y resumen de las operaciones realizadas. 

In [38]:
%load_ext autoreload
%autoreload 2    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
import os
from src.firms_admin_join import setup_logger, run_admin_join, format_report

ruta_municipios = r"D:\Maestria_Geomatica\Semestre_IV\Tesis_Python\Datos\SHP\MunicipiosEPSG4326.shp"
ruta_puntos = r"D:\Maestria_Geomatica\Semestre_IV\Puntos_unificados_25_11_05\puntos_unificados_FIRMS_25_11_24_fecha_desglosada.shp"

carpeta_salida = r"D:\Maestria_Geomatica\Semestre_IV\Puntos_unificados_25_11_05\Puntos_unificados_26_02_23"
out_path = os.path.join(carpeta_salida, "puntos_unificados_FIRMS_municipios_26_02_23.shp")

log_path = os.path.join(carpeta_salida, "log_sjoin_municipios.txt")
logger = setup_logger(log_path)

summary = run_admin_join(
    municipios_path=ruta_municipios,
    puntos_path=ruta_puntos,
    out_path=out_path,
    how="left",
    predicate="intersects",   # puedes cambiar a "within" si lo decides
    version_out=True,
    logger=logger
)

print(format_report(summary))

C:\Users\Equipo\Tesis\SemestreIV\Objetivo1\Objetivo1_26_02_20\src\firms_admin_join.py:121: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(final_path, driver="ESRI Shapefile", encoding=encoding)
C:\Users\Equipo\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field acq_date create as date field, though DateTime requested.
  ogr_write(
C:\Users\Equipo\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field acq_date_n create as date field, though DateTime requested.
  ogr_write(
C:\Users\Equipo\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'index_right' to 'index_righ'
  ogr_write(
C:\Users\Equipo\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'mpnombre_std' to 'mpnombre_s'
  ogr_write(



--- REPORTE FINAL (JOIN MUNICIPIOS) ---
📌 Total puntos: 6744
✅ Asignados (join_ok): 6744
❌ No asignados: 0
🧭 CRS salida: EPSG:4326
💾 Archivo: D:\Maestria_Geomatica\Semestre_IV\Puntos_unificados_25_11_05\Puntos_unificados_26_02_23\puntos_unificados_FIRMS_municipios_26_02_23.shp

